# Churn Modelling

The output of a predictive churn model is a measure of the immediate or future risk of a customer cancelation.
The data set contains details of a bank's customers and the target variable is a binary variable reflecting the fact whether the customer left the bank (closed his account) or he continues to be a customer.
Goal is to predict whether a customer closed his account or continues to be a customer.

In [1]:
#importing the libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
#importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
#checking for missing values
dataset.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [4]:
#creating independent and dependent variables
x = dataset.iloc[:, 3:13]
y = dataset[['Exited']]

In [5]:
#creating a dummy variables
x = pd.get_dummies(x, columns=['Geography', 'Gender'], drop_first=True)

In [6]:
x.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0


In [7]:
#creating train and test data.
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

In [8]:
#scaling the data
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [9]:
#applying Hyper parameter tuning
def create_model(layers, activation_func, k_init) :
    
    model = Sequential()
    
    for i, nodes in enumerate(layers) :
        if i == 0 :
            #adding input layer and 1st hidden layer
            model.add(Dense(units=nodes, kernel_initializer=k_init,
                            activation=activation_func, input_dim=x_train.shape[1]))
            model.add(Dropout(0.3))
        else :
            #adding further hidden layers
            model.add(Dense(units=nodes,
                            kernel_initializer=k_init,
                            activation=activation_func)) 
            model.add(Dropout(rate=0.3))
            
    model.add(Dense(units=1, kernel_initializer=k_init, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [10]:
#applying grid-search
ann_model = KerasClassifier(build_fn=create_model, verbose=0)

layer = [(20,), (40, 20), (45, 30, 15)]
activations = ['sigmoid', 'relu']
initializer = ['he_uniform', 'uniform', 'glorot_uniform']

parameters = (dict(layers=layer, activation_func=activations, k_init=initializer, 
                   batch_size=[30, 80], epochs=[30]))

grid = GridSearchCV(estimator=ann_model, param_grid=parameters, cv=5)

grid_result = grid.fit(x_train, y_train)

In [11]:
print(f'Best score = {round(grid_result.best_score_*100, 3)}%')
print(f'Best parameters = {grid_result.best_params_}')

Best score = 86.014%
Best parameters = {'activation_func': 'relu', 'batch_size': 80, 'epochs': 30, 'k_init': 'glorot_uniform', 'layers': (45, 30, 15)}


In [15]:
y_pred = grid.predict(x_test)
y_pred = (y_pred > 0.5)

In [13]:
#checking the accuracy
print(f"Accuracy of the model = {round(accuracy_score(y_test, y_pred)*100, 3)}%")

Accuracy of the model = 86.033%
